<a href="https://colab.research.google.com/github/Pyfin5/Machine_Learning/blob/main/Compliance_Rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!pip install google-genai langchain-docling langchain-core langchain-huggingface langchain_milvus langchain python-dotenv
!pip install weaviate-client  --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.8/223.8 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.3/254.3 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.2/164.2 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 75.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.7/42.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 55.7 MB/s eta 0:00:00
   ━━━━━━

In [12]:
from google import genai
from google.colab import userdata
from google.genai import types
from google.genai.types import EmbedContentConfig
import os
from docling.document_converter import DocumentConverter
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import PdfPipelineOptions
from docling.document_converter import PdfFormatOption

In [13]:
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Colab-Notebooks/Compliance_RAG/')
print(os.getcwd())
!dir

Mounted at /content/drive
/content/drive/MyDrive/Colab-Notebooks/Compliance_RAG
osfi_resilience.pdf  osfi_tprm.pdf  PayLink_BCP.pdf


In [14]:
tprm_guidelines_path = "osfi_tprm.pdf"
opres_guidelines_path = "osfi_resilience.pdf"

pdf_files = [tprm_guidelines_path, opres_guidelines_path]

# for pdf_path in pdf_files:
#     try:
#         with open(pdf_path, 'rb') as file:
#             # You can now work with the file object 'file' for each PDF
#             print(f"Successfully opened the PDF file: {pdf_path}")


#     except FileNotFoundError:
#         print(f"Error: The file was not found at {pdf_path}")
#     except Exception as e:
#         print(f"An error occurred while processing {pdf_path}: {e}")

In [8]:
pdf_markdown = {'osfi_tprm.pdf': "", 'osfi_resilience.pdf': ""}

for source in pdf_files:
  converter = DocumentConverter()
  result = converter.convert(source)
  print(result)
  print(source)
  doc = result.document

  doc.export_markdown()

KeyboardInterrupt: 

In [32]:
model_kwargs={
                "max_output_tokens": 10000,
                "temperature": 0.2
             }

In [22]:
api_key = userdata.get("GOOGLE_AI_STUDIO_API_KEY")

In [24]:
client = genai.Client(api_key = api_key)


In [25]:
system_prompt = "You are a financnial operational risk expert able to evaluate compliance documents. Use the following schema to evalaute documents provided:"\
                "{{risk function: Business continuity plans,"\
                "{explanation: Business continuity plans set out the response and recovery actions for a range of potential threats. They are"\
                "designed to address disruptions related to people and physical assets, making sure they can function quickly when"\
                "disaster strikes.},"\
                "criteria:{"\
                          "criteria_1:Establishing protocols for invoking the plan and making decisions.,"\
                          "criteria_2:Defining roles and responsibilities for managing disruptions.,"\
                          "criteria_3:Designating backup personnel to cover unexpected absences.,"\
                          "criteria_4:Implementing precautions to ensure safety of staff.,"\
                          "criteria_5:Setting targets for recovery levels and times.,"\
                          "criteria_6:Conducting business impact analysis, workarounds, and recovery strategies."\
                          "criteria_7:Developing internal and external communication plans."\
                          "}"\
                  "}"\
                  "Use each of the criteria provided above to evaluate the document. For each of the criteria defined in the schema, evaluate whether the"\
                  "document contains the given critera. Evaluate each criteria separately and provide an opinion in the same format as above. If evluation is"\
                  "inconclusive, mention 'inconclusive'. Otherwise return 'available' or 'not-available'."\
                  "if the criteria is met, mention the page of the document where the information is found."


In [28]:
document = """IT Business Continuity Plan (BCP)
Company: PayLink Financial Technologies Inc.
Industry: Fintech (Payments Processing)
Version: 1.0
Date: September 3, 2025
Owner: Director of Technology Risk & Compliance1. Introduction & Purpose
PayLink Financial Technologies Inc. ("PayLink") provides digital payment processing, fraud
prevention, and real-time transaction settlement services to banks, merchants, and fintech partners
worldwide. Given the mission-critical nature of our services, any disruption to IT operations poses
significant financial, reputational, and regulatory risks. This IT Business Continuity Plan (BCP)
ensures that PayLink can withstand and recover from disruptions while maintaining service
availability and client trust.
2. Objectives
- Ensure continuity of core payment and transaction systems with minimal disruption.
- Define recovery strategies for IT infrastructure and operations.
- Establish clear roles, responsibilities, and communication protocols.
- Comply with ISO 22301 standards and relevant financial regulations (e.g., PCI DSS, SOC 2).
- Protect client data and maintain trust in PayLink’s services.
3. Business Impact Analysis (BIA) & Risk Assessment
# 3.1 Critical Business Functions
Transaction Processing Engine – RTO: 30m, RPO: 5m
Fraud Detection System – RTO: 1h, RPO: 15m
Customer Support Portal – RTO: 2h, RPO: 30m
Settlement & Reconciliation – RTO: 4h, RPO: 1h
Corporate Email & Collaboration – RTO: 8h, RPO: 12h
# 3.2 Key Risks
- Cybersecurity incidents (ransomware, DDoS attacks)
- Cloud outages (AWS regional failure)
- Data corruption/loss
- Third-party service failure
- Natural disasters
4. Recovery Objectives
- RTO: 30 minutes – 8 hours depending on system criticality
- RPO: 5 minutes – 12 hours depending on data sensitivity5. Recovery Strategies
- Redundant cloud infrastructure across AWS regions
- Database replication and point-in-time recovery
- Automated failover with DNS/load balancing
- Standby ML fraud detection cluster
- Dual data centers for settlement
- Backup vendor contracts
6. Roles & Responsibilities
CIO – approves plan activation
Director, Technology Risk – maintains BCP
Incident Response Manager – activates response team
IT Ops – executes failover
Data Protection Officer – ensures compliance
HR/Comms – manages communications
Vendor Lead – manages third-party providers
7. Communication Protocols
Internal: Slack, SMS, email
External: Client status page, regulator notifications, vendor contacts
Media: PR-controlled statements only
8. Resource Requirements
- Multi-region cloud infra
- Encrypted backups
- VPN-enabled laptops
- $2M annual recovery budget
9. Recovery Procedures
Transaction Outage – failover, validate logs, notify customers
Fraud System Failure – switch vendors, manual review
Data Breach – isolate, investigate, notify, patch
10. Testing, Training & Maintenance- Quarterly tabletop exercises
- Bi-annual failover drills
- Annual full BCP test
- Staff training annually
11. Compliance & Standards
ISO 22301, NIST SP 800-34, PCI DSS, SOC 2, GDPR, FCA, SEC, FINTRAC
12. Plan Maintenance
Reviewed semi-annually by Technology Risk Committee
Last tested: Aug 2025, Next: Feb 2026
Conclusion
PayLink’s BCP ensures resilience, rapid recovery, and regulatory compliance in the face of IT
disruptions."""

In [37]:
user_query = "Evaluate the following document using criteria from the system prompt:"\
            "{document}".format(document=document)


Evaluate the following document using criteria from the system prompt:IT Business Continuity Plan (BCP)
Company: PayLink Financial Technologies Inc.
Industry: Fintech (Payments Processing)
Version: 1.0
Date: September 3, 2025
Owner: Director of Technology Risk & Compliance1. Introduction & Purpose
PayLink Financial Technologies Inc. ("PayLink") provides digital payment processing, fraud
prevention, and real-time transaction settlement services to banks, merchants, and fintech partners
worldwide. Given the mission-critical nature of our services, any disruption to IT operations poses
significant financial, reputational, and regulatory risks. This IT Business Continuity Plan (BCP)
ensures that PayLink can withstand and recover from disruptions while maintaining service
availability and client trust.
2. Objectives
- Ensure continuity of core payment and transaction systems with minimal disruption.
- Define recovery strategies for IT infrastructure and operations.
- Establish clear roles, 

In [38]:
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=document,

    config=types.GenerateContentConfig(
        thinking_config=types.ThinkingConfig(thinking_budget=0), # Disables thinking
        system_instruction=system_prompt,
    ),
)
print(response.text)

```json
{
  "risk function": "Business continuity plans",
  "explanation": "Business continuity plans set out the response and recovery actions for a range of potential threats. They are designed to address disruptions related to people and physical assets, making sure they can function quickly when disaster strikes.",
  "criteria": {
    "criteria_1": {
      "name": "Establishing protocols for invoking the plan and making decisions.",
      "evaluation": "available",
      "page": "6"
    },
    "criteria_2": {
      "name": "Defining roles and responsibilities for managing disruptions.",
      "evaluation": "available",
      "page": "6"
    },
    "criteria_3": {
      "name": "Designating backup personnel to cover unexpected absences.",
      "evaluation": "not-available"
    },
    "criteria_4": {
      "name": "Implementing precautions to ensure safety of staff.",
      "evaluation": "not-available"
    },
    "criteria_5": {
      "name": "Setting targets for recovery levels an